In [71]:
import requests as r
from bs4 import BeautifulSoup
import re
from collections import Counter
import threading
import time


In [72]:
def getLink(page): #把搜尋頁丟近來抓連結的function
    print('thread'+str(page)+'start') #測試多執行緒有沒有在動
    time.sleep(2)
    res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
    time.sleep(0.1)
    soup = BeautifulSoup(res.text,'lxml')
    time.sleep(0.1)
    links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
    time.sleep(0.1)    
    print(len(links))
    time.sleep(0.1)    
    global alinks #拿到外面alinks的值如果沒有加global會變成自己增加區域變數
    alinks += links #alinks = alinks + links 

#     print('thread '+str(page)+' end ')    

In [73]:
class getLinkThread (threading.Thread): #多線程處理
    def __init__(self,page): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self) #繼承父類別（照著打就好）
        self.page=page #this.page=page
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getLink(self.page) #將要爬的頁數丟給getLink去執行


In [74]:
def getWord(link): #把網址丟進來提取文字後做統計的function
    print(link)
    res = r.get(link)
    soup = BeautifulSoup(res.text,'lxml')
    list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
    list2 = str(soup.select('div.content')).upper() #內文
    list3 = list1 + list2 
    words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字
#     time.sleep(0.2)

    for word in words: # 計算所有英文單字的數量
        global wc     
        if word in wc:
            wc[word] += 1
        else:
            wc[word] = 1    

In [75]:
class getWordThread (threading.Thread): #多線程處理
    def __init__(self,link): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self)
        self.link=link
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getWord(self.link)


In [76]:
#第一頁開始：
url='https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&jobcat=2007000000&order=2&asc=0&page=1'
res = r.get(url)
soup = BeautifulSoup(res.text,'lxml')
links = soup.select('div.jobname_summary.job_name > a') #裡面是沒有HOST的
HOST = 'https://www.104.com.tw'
link_list = [HOST + link['href'] for link in links] #把裡面每一個網址都加上HOST
#把link全加到link_list裡面


# 換頁：
buttons = soup.select('span > a')
first_page = 1
# int(buttons[3]['onclick'].split('gopage(')[1].split(');return')[0]) - 2 #3-2
#顯示當前頁面＋當頁所以職缺(一次爬?頁)
#把當頁所有職缺的網址放到alink裡面
alinks = []
page_to_crawl = 10  #要爬幾頁
threads=[]
for page in range(1,151):
    Thread=getLinkThread(page)
    threads.append(Thread)    
for i in threads:
    i.start() #執行緒開始
#     time.sleep(1)
for i in threads:
    i.join()
    
#     print("current page {}".format("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page=".format(page)+"&psl=N_A"))
#     res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
#     soup = BeautifulSoup(res.text,'lxml')
#     links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
#     print(len(links))
#     alinks += links #alinks = alinks + links
#     time.sleep(0.5)


# for i in range(1,10+1): 
#     Thread=MyClass(i) 
#     threads.append(Thread)
# for thread in threads:
#     thread.start() #執行緒開始
# for thread in threads:
#     thread.join()#等同於java中的join 所有執行序跑完在繼續執行下一行指令
#     getLink(page)



#小整合：
wc = Counter() # word count   wc是k:v的形式-> ‘MYSQL': 4’
result_dict = {}
threadwords=[]
for link in alinks:
    threadword=getWordThread(link) 
    threadwords.append(threadword)
for i in threadwords:
    i.start() #執行緒開始
    time.sleep(0.01)
#     time.sleep(0.5)
for i in threadwords:
    i.join()
    
#         res = r.get(link)
#         soup = BeautifulSoup(res.text,'lxml')
#         list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
#         list2 = str(soup.select('div.content')).upper() #內文
#         list3 = list1 + list2
#         words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字

#         for word in words: # 計算所有英文單字的數量
#             if word in wc:
#                 wc[word] += 1
#             else:
#                 wc[word] = 1


#     continue
#     print(link) #有一些廣告或外包網continue
        
string = 'C|C++|C#|PYTHON|JAVA|JAVASCRIPT|PHP|HTML|SQL|CSS|R|BASH|RUBY|PERL|SCALA|SWIFT|GO|DELPHI|TYPESCRIPT|MYSQL'
langs = string.split('|') #string裡面的語言切開

for lang in langs: #每一個語言
    if lang in wc.keys(): #如果有在wc的key裡的話
        result_dict[lang] = wc[lang] #lang是key ;wc[lang]語言數量是value

result_dict

thread1start
thread2startthread3startthread4start


thread5startthread6startthread7startthread8startthread9startthread10startthread11startthread12startthread13startthread14startthread15startthread16startthread17startthread18startthread19startthread20startthread21startthread22startthread23startthread24startthread25startthread26startthread27startthread28startthread29startthread30startthread31startthread32startthread33startthread34startthread35startthread36startthread37startthread38startthread39startthread40startthread41startthread42startthread43startthread44startthread45startthread46startthread47startthread48startthread49startthread50startthread51startthread52startthread53startthread54startthread55startthread56startthread57startthread58startthread59startthread60startthread61startthread62startthread63startthread64startthread65startthread66startthread67startthread68startthread69startthread70startthread71startthread72startthread73startthread74startthread75startthread76startthread77startthre

https://www.104.com.tw/job/?jobno=4v6oc&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4o81f&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=5c0vi&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=53kaa&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=58g34&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4f1hj&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4gxz5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=48h9a&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3o2hh&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3mfkm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3mm7x&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3mqrm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3mqvp&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5g6x1&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=2xwub&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5g4o8&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=50zui&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4rnz1&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=49yjt&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4ases&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4mqbw&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4imkk&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=46xeg&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3wzrv&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3y4zm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3nb9l&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ayo0&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ayo5&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=5fpif&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4lipi&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5c1fj&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5c1fk&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4o4t5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4e7r9&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5gl9t&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5gl9s&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4cmxz&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=3zlv6&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3mgd8&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5f337&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=55d76&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=55d70&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=5arq5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5ejlu&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5ejlv&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5a4pv&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5ejt1&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=5dswl&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5dswz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5dsx1&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4u8hg&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=585ze&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=58ct5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54tc9&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54tcs&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54td4&jobsource=104_bank1&hotj

...

https://www.104.com.tw/job/?jobno=546k8&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5erba&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5a93b&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=55a93&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=57b1s&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=57buw&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5c6zm&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=57qlo&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=57qn0&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4i717&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=481ta&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=2d3zd&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5g51r&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5g53c&jobsource=104_bank1&hotjob

Exception in thread Thread-32865:
Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/models.py", line 374, in prepare_url
    scheme, auth, host, port, path, query, fragment = parse_url(url)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/url.py", line 195, in parse_url
    raise LocationParseError(url)
requests.packages.urllib3.exceptions.LocationParseError: Failed to parse: www.104.com.twjavascript:void(0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-75-cf618ee6c498>", line 6, in run
    getWord(self.link)
  File "<ipython-input-74-dc684331ace4>", line 3, in getWord
    res = r.get(link)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-package

https://www.104.com.tw/job/?jobno=51fic&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=51vfn&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=51xtw&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4xypr&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4trg9&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4o6y0&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4vmvv&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=54bou&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=4vwfk&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4w174&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5gjnb&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5gjn7&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4epoh&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=55qc1&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=4vzro&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4k3sr&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=48jml&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3nsfa&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5ehmu&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5de8e&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5de8k&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5djdj&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5djfa&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5djha&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=58tn7&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=597bc&jobsource=104_bank1&hotjob_chr=


{'BASH': 10,
 'C': 317,
 'C#': 203,
 'C++': 160,
 'CSS': 267,
 'DELPHI': 8,
 'GO': 24,
 'HTML': 417,
 'JAVA': 314,
 'JAVASCRIPT': 783,
 'MYSQL': 211,
 'PERL': 24,
 'PHP': 205,
 'PYTHON': 100,
 'R': 35,
 'RUBY': 20,
 'SCALA': 16,
 'SQL': 312,
 'SWIFT': 34,
 'TYPESCRIPT': 4}